# Modelagem – Housing Califórnia

Treinamento e avaliação de modelos de regressão para previsão do preço médio das casas, incluindo ajuste de hiperparâmetros, cross-validation e cálculo do RMSE.

In [165]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import mean_squared_error
import numpy as np


## 1. Carregamento dos Conjuntos Transformados
Importação das matrizes transformadas de treino (`X_train`) e do alvo (`y_train`) processadas no notebook anterior.

In [166]:
X_train = np.load("data/processed/X_train.npy")
y_train = pd.read_csv("data/processed/y_train.csv").values.ravel() 

## 2. Função de Avaliação
Definição de função para cálculo do RMSE usando cross-validation.

In [167]:
def rmse_cv(model, X, y):
    scores = cross_val_score(model, X, y, scoring="neg_mean_squared_error", cv=5)
    rmse_scores = np.sqrt(-scores)
    return rmse_scores.mean()


## 3. Regressão Linear
Treinamento do modelo de regressão linear e avaliação da performance usando RMSE.

In [168]:
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)

rmse_lin = rmse_cv(lin_reg, X_train, y_train)

print("RMSE Regressão Linear:", rmse_lin)

RMSE Regressão Linear: 69065.59069601324


## 4. Função de Treinamento com Grid Search
Definição de função `train_model` para treinar modelos com Grid Search, ajustar hiperparâmetros e calcular RMSE.

In [169]:
def train_model(model, param_grid, X, y, cv=5):
    grid = GridSearchCV(
        model, 
        param_grid,
        cv=cv,
        scoring='neg_mean_squared_error',
        n_jobs=-1
    )
    grid.fit(X, y)

    best_model = grid.best_estimator_
    scores = cross_val_score(best_model, X, y, scoring='neg_mean_squared_error', cv=cv)
    rmse = np.sqrt(-scores).mean()
    
    return best_model, rmse, grid.best_params_

## 5. Decision Tree Regressor
Treinamento do modelo Decision Tree, ajuste de hiperparâmetros via Grid Search e avaliação do RMSE.

In [170]:
tree = DecisionTreeRegressor(random_state=42)
param_grid_tree = {
    'max_depth': [5, 10, 15, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

best_tree, rmse_tree, best_params_tree = train_model(tree, param_grid_tree, X_train, y_train)
print("Melhores parâmetros Decision Tree:", best_params_tree)
print("RMSE Decision Tree:", rmse_tree)

Melhores parâmetros Decision Tree: {'max_depth': 10, 'min_samples_leaf': 4, 'min_samples_split': 10}
RMSE Decision Tree: 60717.52245459622


## 6. Random Forest Regressor
Treinamento do modelo Random Forest, ajuste de hiperparâmetros via Grid Search e avaliação do RMSE.

In [171]:

rf = RandomForestRegressor(random_state=42)

param_grid_rf = [
    {'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]},
    {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]},
]

best_rf, rmse_rf, best_params_rf = train_model(rf, param_grid_rf, X_train, y_train)
print("Melhores parâmetros Random Forest:", best_params_rf)
print("RMSE Random Forest:", rmse_rf)

Melhores parâmetros Random Forest: {'max_features': 8, 'n_estimators': 30}
RMSE Random Forest: 49893.91429940498


## 7. Salvamento dos Modelos
Exportação dos modelos finais treinados (`Decision Tree`, `Random Forest` e `Linear Regression`) usando `joblib` para uso na avaliação.

In [172]:
import joblib
import os

os.makedirs("models", exist_ok=True)

joblib.dump(best_tree, "models/decision_tree_model.pkl")
joblib.dump(best_rf, "models/random_forest_model.pkl")
joblib.dump(lin_reg, "models/linear_regression_model.pkl")

['models/linear_regression_model.pkl']